<a href="https://colab.research.google.com/github/HYChangTAMU/Financial-Data-Analysis/blob/main/Assignment5/Project5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Date 11/26/2023

#-------------------------------------------------------------------------------

!pip install mat73
!pip install ZCAPM
import mat73
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import json
import datetime
import pytz
import ZCAPM
from statsmodels.stats.weightstats import ttest_ind, ttost_paired
from sklearn.metrics import r2_score
from google.colab import files


#-------------------------------------------------------------------------------

f6 = pd.read_csv('/content/drive/MyDrive/Assignment4/data/6_factors_daily.csv')
f6.drop(range(14979, 15123), inplace = True)


#-------------------------------------------------------------------------------

daily_return = pd.DataFrame(mat73.loadmat('/content/drive/MyDrive/Assignment4/data/stock_daily_return.mat')['stock_ret'])
daily_return[0] = daily_return[0].astype('int64')
daily_return.rename({0: 'Date'}, axis=1, inplace=True)
daily_return.drop(range(879), inplace = True)
daily_return.reset_index(drop = True, inplace = True)


#-------------------------------------------------------------------------------

# start and end of 714 months
# month_start_end records the dates
# month_start_end_index records the indices

month_start_end = []
dates = f6['Date'][0:-1].tolist()
for i, d in enumerate(dates):
  if d%100 < dates[i-1]%100:
    month_start_end.append(dates[i-1])
    month_start_end.append(d)
month_start_end.append(month_start_end[0])
month_start_end = month_start_end[1:]

mse_index = []
for i in range(len(month_start_end)//2):
  mse_index.append(f6['Date'].index[f6['Date'] == month_start_end[2*i]].tolist())
  mse_index.append(f6['Date'].index[f6['Date'] == month_start_end[2*i + 1]].tolist())

month_start_end_index = []
for i in range(len(month_start_end)):
  month_start_end_index.append(mse_index[i][0])
month_start_end[-1] = 20221230
month_start_end_index[-1] = 14978

## Testing the ZCAPM Package


In [10]:
test = ZCAPM.Testing()

In [11]:
#Set Parameters for rolling estimations and EM Algorithm. For information on parameters see doc string of _EM_loop method
tol = .001
MaxIter = 1000
criterion = 1
width = 3

######## IMPORTANT########
#trims off all of the first "width" amount of monthly returns to ensure that the Fama-MacBeth test is performed
#OUT OF SAMPLE
monthly_excess_return = test.sorted_portfolio_monthly_excess_return.iloc[width:,:]
portfolio_excess_return = test.sorted_portfolio_daily_excess_return
factor_return = test.factor_daily_return

YearMonth = monthly_excess_return.index

#create pandas series for mkt ret, mkt sigma, and factors. Convert indices of these series and portfolio return dataframe
#to be the YearMonth list. Useful for indexing purposes while testing
mu = (factor_return.loc[:,"R_a.R_f"])
sigma = (factor_return.loc[:,"sigma_a"])
facs_ret = factor_return.loc[:,['YearMonth','R_a.R_f','SMB','HML']]

mu.index = factor_return.YearMonth
sigma.index = factor_return.YearMonth
portfolio_excess_return.set_index('YearMonth',inplace = True)
facs_ret.set_index('YearMonth',inplace =True)

In [24]:
factor_return

,YearMonth,R_a.R_f,sigma_a,R_f,SMB,HML,MOM
Date,,,,,,,
1964-01-02,196401,0.60,1.51160,0.013,0.61,0.57,-0.09
1964-01-03,196401,0.17,1.44340,0.013,0.11,0.39,-0.38
1964-01-06,196401,0.23,1.42870,0.013,0.14,0.01,-0.16
1964-01-07,196401,0.04,1.46680,0.013,0.08,0.76,-0.38
1964-01-08,196401,0.34,1.22780,0.013,0.04,-0.15,0.01
...,...,...,...,...,...,...,...
2015-12-24,201512,-0.11,0.76288,0.000,0.30,-0.08,0.08
2015-12-28,201512,-0.29,1.39370,0.000,-0.49,-0.35,1.41
2015-12-29,201512,1.05,0.91048,0.000,0.09,-0.31,0.20


In [12]:
#calculates time series factor loadings for each portfolio. See Testing class for information on the arguments of each
#method
zcapm_results = test.estZCAPM(portfolio_excess_return,mu,sigma,tol,MaxIter,criterion,width)
ff3_results = test.estLinearModel(portfolio_excess_return,facs_ret,width)
capm_results = test.estLinearModel(portfolio_excess_return,facs_ret.loc[:,['R_a.R_f']],width)

#adjust the zeta estimates for each portfolio to monthly estimates
zeta_cols = zcapm_results.columns[zcapm_results.columns.str.contains('zeta')]
zcapm_results.loc[:,zeta_cols] = zcapm_results.loc[:,zeta_cols]*21

In [13]:
#Runs the Fama-MacBeth Test for each portfolio
ZCAPM = test.FamaMacBeth(monthly_excess_return,zcapm_results,['beta','zeta'],'ZCAPM')
FF3 = test.FamaMacBeth(monthly_excess_return,ff3_results,['R_a.R_f','SMB','HML'],'Fama-French 3 Factor')
CAPM = test.FamaMacBeth(monthly_excess_return,capm_results,['R_a.R_f',],'CAPM')

In [14]:
ZCAPM

,coefficients,t-values
ZCAPM,,
intercept,0.7512296664738565,3.497781612467159
beta,-0.1637109804339669,-0.7902428749234214
zeta,0.2777671477401986,4.440021124542478
Single Regression Approach R-squared,0.9434936089919264,


In [15]:
FF3

,coefficients,t-values
Fama-French 3 Factor,,
intercept,0.8301745308930485,4.662110778011936
R_a.R_f,-0.3103320201874723,-1.6554497522919884
SMB,0.18693279566558293,1.4866973964195755
HML,0.2767534942772498,2.422771807333926
Single Regression Approach R-squared,0.6410808926753392,


In [16]:
CAPM

,coefficients,t-values
CAPM,,
intercept,0.9311867580090324,4.098364634696492
R_a.R_f,-0.3221253240213647,-1.4438116059978983
Single Regression Approach R-squared,0.5234846086050577,
